In [0]:
CREATE OR REPLACE VIEW case_study_de.profiles_data_days_2_3 AS (WITH end_of_2 AS 
(SELECT incremental_date FROM sequential_date ORDER BY incremental_date LIMIT 2),
active_customers_day_2 AS 
(SELECT DISTINCT customer_nk_num, customer_version FROM order_bronze WHERE order_date IN (SELECT * FROM end_of_2)),
active_customers_day_3_diff AS 
(SELECT DISTINCT customer_nk_num, customer_version FROM order_bronze WHERE order_date IN (SELECT (MAX(*) + INTERVAL 1 DAY) FROM end_of_2)),
discontinued_profiles AS 
(SELECT 
cb.customer_nk_num,
cb.customer_version, 
2 AS until_day, 
ac.customer_nk_num AS nk_num_order
FROM customer_bronze cb
LEFT JOIN active_customers_day_2 ac 
ON cb.customer_nk_num = ac.customer_nk_num AND cb.customer_version = ac.customer_version
UNION
SELECT 
cb.customer_nk_num,
cb.customer_version, 
3 AS until_day, 
ac.customer_nk_num AS nk_num_order
FROM customer_bronze cb
LEFT JOIN active_customers_day_3_diff ac 
ON cb.customer_nk_num = ac.customer_nk_num AND cb.customer_version = ac.customer_version)
SELECT * FROM discontinued_profiles ORDER BY until_day)